## create a connection to the database using the library sqlite3

In [12]:
import pandas as pd
import sqlite3
connection = sqlite3.connect('../data/checking-logs.sqlite')

## create a new table datamart in the database by joining the tables pageviews and checker

In [13]:

query =  """SELECT checker.uid, labname, timestamp AS 'first_commit_ts',
            MIN(pageviews.datetime) AS 'first_view_ts' FROM checker
            LEFT JOIN pageviews ON checker.uid=pageviews.uid
            WHERE status='ready' AND numTrials=1 AND checker.uid LIKE 'user_%'
            AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
            GROUP BY timestamp"""
datamart = pd.io.sql.read_sql(query, connection, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## create two dataframes: test and control

In [14]:
test = datamart[datamart['first_view_ts'].notnull()]
test.head(10)

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [15]:
control = datamart[datamart['first_view_ts'].isnull()]
control.head(10)

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
5,user_2,laba04,2020-04-18 13:42:35.482008,NaT
6,user_2,laba04s,2020-04-18 13:51:22.291271,NaT
9,user_12,laba04,2020-04-18 17:07:51.767358,NaT
10,user_8,laba04,2020-04-18 21:53:36.058389,NaT
12,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
13,user_29,laba04,2020-04-19 11:10:55.722647,NaT
14,user_29,laba04s,2020-04-19 18:55:41.516438,NaT


In [16]:
control = control.fillna(test['first_view_ts'].mean())
control.head(10)

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
9,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
10,user_8,laba04,2020-04-18 21:53:36.058389,2020-04-27 00:40:05.761783552
12,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
13,user_29,laba04,2020-04-19 11:10:55.722647,2020-04-27 00:40:05.761783552
14,user_29,laba04s,2020-04-19 18:55:41.516438,2020-04-27 00:40:05.761783552


In [18]:
test.to_sql('test', connection)
control.to_sql('control', connection)

81

## close the connection

In [ ]:
connection.close()